[Jefferson-Madison Regional Library](http://jmrl.org) is located in central Virginia.  Its nine branch locations serve a population of over 200,000 residents of Albemarle, Greene, Louisa, and Nelson counties and the city of Charlottesville.

In [1]:
import pandas as pd
import geopandas as gpd
import folium

In [2]:
'''
utility definitions and functions
'''
DATA_DIR = 'https://raw.githubusercontent.com/roadfoodr/JMRL-notebooks/master/data/'

COUNTYFP_CODES = {
        '003': 'Albemarle',
        '540': 'Charlottesville',
        '079': 'Greene',
        '109': 'Louisa',
        '125': 'Nelson',
        }

JMRL_BOUNDS = [[37.535641999999996, -79.172135], [38.475527, -77.687294]]

In [3]:
# read data
branches = gpd.read_file(f'{DATA_DIR}branches.geojson')
df_counties = gpd.read_file(f'{DATA_DIR}counties.geojson')

HTTPError: HTTP Error 404: Not Found

In [ ]:
# df_counties.dtypes

In [ ]:
# gpd.__version__
# folium.__version__
map_bounds = JMRL_BOUNDS

In [ ]:
%%capture
m = folium.Map(location=None,
#               tiles='Stamen Toner',
               tiles=None,
#                zoom_start=9
              )

m.fit_bounds(map_bounds,
             padding_top_left=(0,0), padding_bottom_right=(0,0)
            )

folium.TileLayer(
        tiles='https://stamen-tiles-{s}.a.ssl.fastly.net/toner-lite/{z}/{x}/{y}{r}.png',
        attr=f'Map tiles by <a href="http://stamen.com">Stamen Design</a>, '
        f'<a href="http://creativecommons.org/licenses/by/3.0">CC BY 3.0</a> &mdash; '
        f'Map data &copy; <a href="https://www.openstreetmap.org/copyright">OpenStreetMap</a> '
        f'contributors',
        control=False).add_to(m)

In [ ]:
%%capture
county_layer = folium.Choropleth(
        geo_data=df_counties,
        name='County outlines',
        data=df_counties,
        columns = ['COUNTYFP', 'POPESTIMATE2019'],
        key_on='feature.properties.COUNTYFP',
        legend_name='County Population (2019 Estimate)',
        fill_color='YlGn',
        fill_opacity=0.5, line_opacity=0.4,
        smooth_factor = 1.66,
        highlight=True,
        )

folium.GeoJsonTooltip(fields=['NAMELSAD', 'POPESTIMATE2019'],
                     aliases=['', 'Pop.']).add_to(county_layer.geojson)

county_layer.add_to(m)

# folium.LayerControl(collapsed=False).add_to(test_map)

In [ ]:
%%capture
branch_layer = folium.FeatureGroup(name='Library Branches')
for i in range(0,len(branches)):
    folium.Marker(
        location=[branches.iloc[i]['lat'], branches.iloc[i]['long']],
        tooltip=f"{branches.iloc[i]['id']} Branch<br>" \
        + f"{branches.iloc[i]['addr_combined']}",
        icon=folium.Icon(prefix='fa', icon='book',
                         markerColor='blue', iconColor='#fa7906')
        ).add_to(branch_layer)

branch_layer.add_to(m)

In [ ]:
display(m)